In [1]:
import pandas as pd
import requests
from io import StringIO
import time
from bs4 import BeautifulSoup

# 展開報表結果
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

def monthly_report(year, month, trace_month, companies, is_first):
    
    # 追蹤月數
    trace_month = trace_month - 1
    
    # 假如是西元，轉成民國
    if year > 1990:
        year -= 1911
    
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if year <= 98:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'
    
    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # 下載該年月的網站，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big5')
    
    # 取出column數大於5小於11之資料
    df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])
    
    # 將雙層colume簡化為一層
    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1).str.replace(r'\W','',regex=True)
    else:
        df = df[list(range(0,10))]
        column_index = df.index[(df[0] == '公司代號'.decode('utf-8'))][0]
        df.columns = df.iloc[column_index]
        
    df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
    df = df[~df['當月營收'].isnull()]
    df = df[df['公司代號'] != '合計']
    
    new_companies = []
    if len(companies) > 0:
        df = df[df['公司代號'].isin(companies)]
    elif len(companies) == 0 and is_first == False :
        result = ["沒有資料"]
        result_dict = {"result":result}
        df = pd.DataFrame(result_dict)
        return df
    
    df = df[pd.to_numeric(df['去年同月增減'], errors='coerce') > 0]
    new_companies = df['公司代號'].values
    
    if trace_month >= 1 :
        is_first = False
        month = month - 1
        if month == 0 :
            month = 12
            year = year - 1
        df = monthly_report(year, month, trace_month, new_companies, is_first)
    else :
        return df
    
    return df

In [ ]:
year = 2023
month = 7
trace_month = 6
companies = []
df = monthly_report(year, month, trace_month, companies, is_first=True)
df = df.reset_index(drop=True)
df

In [ ]:
year = 2023
month = 6
trace_month = 6
companies = []
df2 = monthly_report(year, month, trace_month, companies, is_first=True)
df2 = df2.reset_index(drop=True)
df2

In [4]:
df3 = df2[~df2['公司代號'].isin(df['公司代號'])]
df3 = df3.reset_index(drop=True)
df3.to_csv('C:/Users/Skyluck/Desktop/差異減少.csv', encoding="utf_8_sig")

df4 = df[~df['公司代號'].isin(df2['公司代號'])]
df4 = df4.reset_index(drop=True)
df4.to_csv('C:/Users/Skyluck/Desktop/差異新增.csv', encoding="utf_8_sig")